In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string

In [2]:
fake=pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
fake.shape

(23481, 4)

In [5]:
true.shape

(21417, 4)

In [6]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
fake['class']=0
true['class']=1

In [8]:
fake_train=fake.head(18000)
true_train=true.head(18000)

In [9]:
fake.drop(fake_train.index,axis='rows',inplace=True)
true.drop(true_train.index,axis='rows',inplace=True)

In [10]:
print(fake.shape)
print(true.shape)

(5481, 5)
(3417, 5)


In [11]:
test=pd.concat([fake,true],axis=0)

In [12]:
train=pd.concat([true_train,fake_train],axis=0)
train.shape

(36000, 5)

In [13]:
train.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [14]:
for i in range(10):
    train=train.sample(n=30000,replace=True).reset_index(drop=True)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    30000 non-null  object
 1   text     30000 non-null  object
 2   subject  30000 non-null  object
 3   date     30000 non-null  object
 4   class    30000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [16]:
tr=train.drop(['title','date','subject'],axis=1)
test=test.drop(['title','date','subject'],axis=1)

In [17]:
import re

In [18]:
def worddrop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [19]:
tr['text']=tr['text'].apply(worddrop)
test['text']=test['text'].apply(worddrop)

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import porter

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stops=stopwords.words('english')

In [22]:
tr['text'] = tr['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
test['text']= test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))


In [23]:
stemmer=porter.PorterStemmer()

In [24]:
tr['text']=tr['text'].apply(lambda x:' '.join([stemmer.stem(words) for words in x.split()]))
test['text']=test['text'].apply(lambda x:' '.join([stemmer.stem(words) for words in x.split()]))


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

count_v=CountVectorizer(max_features=5000)
tf_idf=TfidfVectorizer(max_features=5000)
one_h=OneHotEncoder(max_categories=100)

In [26]:
y=tr['class']
tr=tr.text

test_new=test.text
test_y=test['class']



count_v.fit(tr)
tf_idf.fit(tr)

x_cv=count_v.transform(tr)
x_tf=tf_idf.transform(tr)

x_cv_test=count_v.transform(test_new)
x_tf_test=tf_idf.transform(test_new)

In [27]:
x_cv.shape

(30000, 5000)

In [28]:
tr_train,tr_test,y_train,y_test=train_test_split(x_cv,y,test_size=0.2,random_state=8)

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
ml=MultinomialNB()
ml.fit(tr_train,y_train)


pa=PassiveAggressiveClassifier()
pa.fit(tr_train,y_train)

lg=LogisticRegression()
lg.fit(tr_train,y_train)

rmf=RandomForestClassifier()
rmf.fit(tr_train,y_train)

gb=GradientBoostingClassifier()
gb.fit(tr_train,y_train)

GradientBoostingClassifier()

In [31]:
ml_pred=ml.predict(tr_test)
print(accuracy_score(ml_pred,y_test))

pa_pred=pa.predict(tr_test)
print(accuracy_score(pa_pred,y_test))

lg_pred=lg.predict(tr_test)
print(accuracy_score(lg_pred,y_test))

rmf_pred=rmf.predict(tr_test)
print(accuracy_score(rmf_pred,y_test))

gb_pred=gb.predict(tr_test)
print(accuracy_score(gb_pred,y_test))


0.9446666666666667
0.9993333333333333
0.9998333333333334
0.9998333333333334
0.9996666666666667


In [32]:
test_pred_ml=ml.predict(x_cv_test)
test_pred_pa=pa.predict(x_cv_test)
test_pred_lg=lg.predict(x_cv_test)
test_pred_rmf=rmf.predict(x_cv_test)
test_pred_gb=gb.predict(x_cv_test)




In [33]:
print(accuracy_score(test_pred_ml,test_y))
print(accuracy_score(test_pred_pa,test_y))
print(accuracy_score(test_pred_lg,test_y))
print(accuracy_score(test_pred_rmf,test_y))
print(accuracy_score(test_pred_gb,test_y))

0.9006518318723309
0.9779725781074399
0.9843785120251742
0.9884243650258485
0.9893234434704428


In [34]:
tr_train,tr_test,y_train,y_test=train_test_split(x_tf,y,test_size=0.2,random_state=8)

In [35]:
ml=MultinomialNB()
ml.fit(tr_train,y_train)


pa=PassiveAggressiveClassifier()
pa.fit(tr_train,y_train)

lg=LogisticRegression()
lg.fit(tr_train,y_train)

rmf=RandomForestClassifier()
rmf.fit(tr_train,y_train)

gb=GradientBoostingClassifier()
gb.fit(tr_train,y_train)

GradientBoostingClassifier()

In [36]:
ml_pred=ml.predict(tr_test)
print(accuracy_score(ml_pred,y_test))

pa_pred=pa.predict(tr_test)
print(accuracy_score(pa_pred,y_test))

lg_pred=lg.predict(tr_test)
print(accuracy_score(lg_pred,y_test))

rmf_pred=rmf.predict(tr_test)
print(accuracy_score(rmf_pred,y_test))

gb_pred=gb.predict(tr_test)
print(accuracy_score(gb_pred,y_test))

0.9351666666666667
0.9991666666666666
0.9941666666666666
0.9998333333333334
0.9996666666666667


In [37]:

test_tf=tf_idf.transform(test_new)

test_pred_ml=ml.predict(x_tf_test)
test_pred_pa=pa.predict(x_tf_test)
test_pred_lg=lg.predict(x_tf_test)
test_pred_rmf=rmf.predict(x_tf_test)
test_pred_gb=gb.predict(x_tf_test)



In [38]:
print(accuracy_score(test_pred_ml,test_y))
print(accuracy_score(test_pred_pa,test_y))
print(accuracy_score(test_pred_lg,test_y))
print(accuracy_score(test_pred_rmf,test_y))
print(accuracy_score(test_pred_gb,test_y))

0.8952573612047651
0.9772982692739941
0.9675207911890312
0.9893234434704428
0.9880872106091256
